In [43]:
import pandas as pd
import glob
import numpy as np

csv_dir_path = "../WebScraper/seedly-forum/*.csv"

In [24]:
#method for cleanning up username string

def split_user_details(user_details):
    user_details = user_details.replace("Answered on","")
    user_details = user_details.replace("Updated on","")
    commar_index = user_details.find(",")
    level_index = user_details.find("Level")
    level_rank = 0
    # 
    if level_index != -1:
        level_rank = user_details[level_index:]
    # commar delimeter applicable to some username string but not all.
    if commar_index != -1:
        username = user_details[:commar_index+1]
    else:
        username = user_details[:level_index+1]
    
    # Remove this particular string from usernames if it exist
    username = username.replace("Top Contributor (Sep)","")
    
    # Remove comments string from usernames if it exists
    comments_index = username.find("comments")
    if comments_index != -1:
        username = username[comments_index+9:]
    if username.endswith("L"):
        username = username[:-1]
    username = username.replace(",","")
    
    # Check if user exists as a user in seedly. If not give them default values
    if level_rank!=0:
        dot_substring = level_rank.find(".")
        level_substring = level_rank.find("Level")
        level = int(level_rank[level_substring+6:dot_substring])
        rank = level_rank[dot_substring+2:]
    else:
        level = 0
        rank = "Nub"
    
    return username,level,rank
        

In [25]:
# Returns the dict where username has already existed in the network
def get_repeated_user_dict(post_answer_list,username):
    
    for i, answer_dict in enumerate(post_answer_list):
        if username == answer_dict.get("username"):
            return post_answer_list.pop(i), post_answer_list

In [26]:
# Code cleans and returns dictionary of cleaned answer and question
# It is assumed one forum post only has one Anonymous User

import re

question_list = []
answer_list = []
check_answer_list = []
check_question_list = []
# Regex for date identification within comments
regex = r'\d{2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}'
qid = 0
annon_counter = 1

# clean each file individually and store each line of question and answer individiually into a list
for filename in glob.glob(csv_dir_path):
    file_df = pd.read_csv(filename)
    qid += 1
    
    # Check if username did change for Anonymous users in each post
    check_username_change = False
    # Used to check if username exists within post
    check_username_list = []
    #accumulate non-repeated user in post
    post_answer_list = []
    #
    for i,content in enumerate(file_df.content):
        user_details = ""
        question_dict = {}
        answer_dict = {}
        answer = ""
        username = ""
        # First row is always question
        if i == 0:
            substr_index = content.find("Asked by")
            q_user_details = content[substr_index:]
            new_sub_index = q_user_details.find("Asked on")
            question_name = q_user_details[:new_sub_index].replace("Asked by","").strip()
            if question_name.find("Asked")!= -1:
                question_name = question_name[:question_name.find("Asked")]
            question = q_user_details[new_sub_index+20:]
            question = question[:question.find("answer")-2]
            question_dict["post_id"] = qid
            question_dict["name"] = question_name
            question_dict["question"] = question
        
        # when i == 1 row is just a header row. Afterwards is the actual answers
        elif i > 1:
            #Extract Answer and user details
            substr_index = content.find("Answered")
            pos_substr_index = content.find("Updated")
            date = re.findall(regex,content)
            
            #Check using date substring first
            if date:
                date_substr = content.find(date[0])
                user_details = content[:date_substr]
                answer = content[date_substr+11:]
                answer = answer[:answer.find("comments")-2]
                
            # Check if Answered string is a substring
            elif substr_index != -1:
                answer = content[substr_index:]
                answer = answer[:answer.find("comments")-2]
                w_substr = answer.find("w ago")
                h_substr = answer.find("h ago")
                d_substr = answer.find("d ago")
                if w_substr!= -1:
                    answer = answer[w_substr+5:]
                elif h_substr!= -1:
                    answer = answer[h_substr+5:]
                elif d_substr != -1:
                    answer = answer[d_substr+5:]
                user_details = content[:substr_index]
                
            # Check if Updated String is a substring    
            elif pos_substr_index!= -1:
                answer = content[pos_substr_index:]
                answer = answer[:answer.find("comments")-2]
                w_substr = answer.find("w ago")
                h_substr = answer.find("h ago")
                d_substr = answer.find("d ago")
                if w_substr!= -1:
                    answer = answer[w_substr+5:]
                elif h_substr!= -1:
                    answer = answer[h_substr+5:]
                elif d_substr != -1:
                    answer = answer[d_substr+5:]
                user_details = content[:pos_substr_index]
                
            # Filtered words don't exist. Straight away remove using period taken since user has answered post
            else:
                w_substr = content.find("w ago")
                h_substr = content.find("h ago")
                d_substr = content.find("d ago")
                if w_substr!= -1:
                    answer = content[w_substr+5:]
                    user_details = content[:w_substr-1]
                elif h_substr!= -1:
                    answer = content[h_substr+5:]
                    user_details = content[:h_substr-1]
                elif d_substr != -1:
                    answer = content[d_substr+5:]
                    user_details = content[:d_substr-1]
                    
        # Check if user_details has been populated before if not call method to generate default user
        if user_details:
            username, level, rank = split_user_details(user_details)
        
        # Check if is Anonymous user. If not create default user for post
        if not username:
            username = "Anonymous " + str(annon_counter)
            if not check_username_change:
                annon_counter += 1
                check_username_change = True
        
        # Check if answer exists to remove repeated answers from repeated content crawled based on repeated crawled content
        if answer and answer not in check_answer_list:
            check_answer_list.append(answer)
            username = username.replace("DeleteEdit","")
            if username not in check_username_list and "Anonymous" not in username:
                check_username_list.append(username)
                answer_dict["post_id"] = qid
                answer_dict["username"] = username
                answer_dict["username_count"] = 1
                answer_dict["level"] = level
                answer_dict["rank"] = rank
                answer_dict["answer"] = answer
                post_answer_list.append(answer_dict)
            elif "Anonymous" in username:
                continue
            else:
                new_answer_dict, post_answer_list = get_repeated_user_dict(post_answer_list,username)
                new_answer_dict["username_count"] += 1
                post_answer_list.append(new_answer_dict)
        
        # Check if question exists to remove repeated question from repeated content crawled based on repeated crawled content
        if question_dict and question and question not in check_question_list:
            question_list.append(question_dict)
            check_question_list.append(question)
            
    answer_list.extend(post_answer_list)

        

# General storing and EDA of data cleaned

In [27]:
answer_df = pd.DataFrame(answer_list)

In [28]:
question_df = pd.DataFrame(question_list)

In [29]:
answer_df

,answer,level,post_id,rank,username,username_count
0,To 'hang out with friends and go out occasiona...,8,1,Wizard,Junus Eu,1
1,Budgeting is key! Before you spend you hard-ea...,2,1,Rookie,Good Samaritan,1
2,Go somewhere cheaper? if not just skip it. The...,8,1,Wizard,Gabriel Tham,1
3,"Dear Anonymous,It's difficult to time the mark...",5,2,Genius,Sin Ting So,1
4,I would suggest to stick with your risk alloca...,8,2,Wizard,Gabriel Tham,1
5,I am using it to send money back to Malaysia f...,2,3,Rookie,Lim ChoonGuan,1
6,Hey Anonymous! All remittance services carry s...,2,3,Rookie,Jonathan Lim,1
7,"Hi,The pro is that you can get a feel on the v...",4,4,Prodigy,Victor Chng,1
8,Another thing is that RVs don't use your predi...,6,4,Master,Richard Woon Tian Jun,1
9,Here are some cons that you should look out fo...,8,4,Wizard,Isaac Chan,1


In [30]:
question_df

,name,post_id,question
0,Anonymous,1,I am trying to save money and at the same time...
1,Anonymous,2,So I have an investment in Robo Advisor since ...
2,Anonymous,3,Has anyone used InstaReM before?I'm looking to...
3,Anonymous,4,What are some pros and cons of using relative ...
4,Kenneth Chia,5,Does anyone buy China stocks/bonds direct? How...
5,Tan Wei Jie Shawn,6,Do you guys think that DCA or LSI in an ETF is...
6,Anonymous,7,"As a start-up, how much do you guys pay your i..."
7,Shabir Khan,8,Are there any ETFs' in Singapore that re-inves...
8,Anonymous,9,"What are the differences between SPY, VOO, IVV..."
9,Anonymous,10,Which accounting courses or books would you re...


In [31]:
answer_df[answer_df["level"] >= 6].groupby("username").head()

,answer,level,post_id,rank,username,username_count
0,To 'hang out with friends and go out occasiona...,8,1,Wizard,Junus Eu,1
2,Go somewhere cheaper? if not just skip it. The...,8,1,Wizard,Gabriel Tham,1
4,I would suggest to stick with your risk alloca...,8,2,Wizard,Gabriel Tham,1
8,Another thing is that RVs don't use your predi...,6,4,Master,Richard Woon Tian Jun,1
9,Here are some cons that you should look out fo...,8,4,Wizard,Isaac Chan,1
15,Either way works.​You can DCA with a lump sum ...,8,6,Wizard,Gabriel Tham,1
16,"Statistically, its always Lump Sum investments...",6,6,Master,Luke Ho,1
18,I would do DCA too since I don't have to time ...,6,6,Master,Gabriel Lee,1
22,This is a good question and I will reply from ...,8,7,Wizard,Kenneth Lou,1
24,"As far as i know, there is none. You have to r...",6,8,Master,Nicholes Wong,1


# Networkx Analysis

In [32]:
import networkx as nx
import matplotlib.pyplot as plt

In [33]:
user_answer_df = answer_df.drop("answer",axis=1)
user_answer_df

,level,post_id,rank,username,username_count
0,8,1,Wizard,Junus Eu,1
1,2,1,Rookie,Good Samaritan,1
2,8,1,Wizard,Gabriel Tham,1
3,5,2,Genius,Sin Ting So,1
4,8,2,Wizard,Gabriel Tham,1
5,2,3,Rookie,Lim ChoonGuan,1
6,2,3,Rookie,Jonathan Lim,1
7,4,4,Prodigy,Victor Chng,1
8,6,4,Master,Richard Woon Tian Jun,1
9,8,4,Wizard,Isaac Chan,1


In [35]:
G = nx.from_pandas_edgelist(user_answer_df, source="username",target="post_id",edge_attr=["username_count"])
G.number_of_nodes()

6987

In [36]:
G.number_of_edges()

15005

In [37]:
n = 10

# compute degree centrality
degree_centrality = nx.degree(G)
# highest degree centrality
top_n_dc = sorted(degree_centrality, key=lambda x: x[1], reverse=True)[:n]

for author, d in top_n_dc:
    print(d,author)

825 Gabriel Tham
822 Hariz Arthur Maloy
480 Jonathan Chia Guangrong
454 Yong Kah Hwee
407 Nicholes Wong
384 Kenneth Lou
350 Junus Eu
350 Loh Tat Tian
342 HC Tang
327 Lok Yang Teng


In [38]:
from collections import OrderedDict
# compute normalised degree centrality
normalized_degree_centrality = nx.degree_centrality(G)
# highest normalised degree centrality
top_n_ndc = OrderedDict(sorted(normalized_degree_centrality.items(), key=lambda x: x[1], reverse=True)[:n])

for author in top_n_ndc:
    print(top_n_ndc[author],"\t",author)

0.1180933295161752 	 Gabriel Tham
0.11766389922702547 	 Hariz Arthur Maloy
0.06870884626395649 	 Jonathan Chia Guangrong
0.0649871170913255 	 Yong Kah Hwee
0.05825937589464643 	 Nicholes Wong
0.05496707701116518 	 Kenneth Lou
0.0501002004008016 	 Junus Eu
0.0501002004008016 	 Loh Tat Tian
0.04895505296306899 	 HC Tang
0.04680790151732035 	 Lok Yang Teng


In [18]:
user_answer_df.to_csv("users_seedly_network.csv",encoding="utf-8")

In [39]:
user_answer_df.head()

,level,post_id,rank,username,username_count
0,8,1,Wizard,Junus Eu,1
1,2,1,Rookie,Good Samaritan,1
2,8,1,Wizard,Gabriel Tham,1
3,5,2,Genius,Sin Ting So,1
4,8,2,Wizard,Gabriel Tham,1


# Generate GML

In [47]:
# sum weights
consolidated_user_answer = {}
for index, row in user_answer_df.iterrows():
    key = (row['username'],row['post_id'])

    if key in consolidated_user_answer:
        consolidated_user_answer[key] += 1
    else:
        consolidated_user_answer[key] = 1
consolidated_user_answer

{('Junus Eu', 1): 1,
 ('Good Samaritan', 1): 1,
 ('Gabriel Tham', 1): 1,
 ('Sin Ting So', 2): 1,
 ('Gabriel Tham', 2): 1,
 ('Lim ChoonGuan', 3): 1,
 ('Jonathan Lim', 3): 1,
 ('Victor Chng', 4): 1,
 ('Richard Woon Tian Jun', 4): 1,
 ('Isaac Chan', 4): 1,
 ('Alan Seow', 5): 1,
 ('Goh Kah Kiat', 5): 1,
 ('Jefferson Soh', 5): 1,
 ('Lee Jiahui', 6): 1,
 ('Boonhow Eng', 6): 1,
 ('Gabriel Tham', 6): 1,
 ('Luke Ho', 6): 1,
 ('Marcus Goh', 6): 1,
 ('Gabriel Lee', 6): 1,
 ('Phyllis Poon', 6): 1,
 ('Jerry Tong', 6): 1,
 ('Hannah Foo', 6): 1,
 ('Kenneth Lou', 7): 1,
 ('Rachel Yeo', 7): 1,
 ('Nicholes Wong', 8): 1,
 ('Harvey Tan', 9): 1,
 ('Kenneth Lou', 10): 1,
 ('Gabriel Tham', 10): 1,
 ('Lim Boon Tat', 10): 1,
 ('Douglas Chow', 11): 1,
 ('Yixiong Chang', 11): 1,
 ('Good Day Every Day', 12): 1,
 ('Zeeeee', 12): 1,
 ('HC Tang', 12): 1,
 ('Brandan Chen', 12): 1,
 ('Yeap Ming Feng', 12): 1,
 ('Kenneth Lou', 12): 1,
 ('Kenneth Chan', 13): 1,
 ('Gabriel Tham', 13): 1,
 ('Gabriel Tham', 14): 1,
 ('Youn

In [48]:
# Start writing a GML output file
output_file = open("seedly_forum_network.gml", "w")
output_file.write("graph\n")
output_file.write("[\n")

2

In [49]:
# print all nodes
id_count = 10000
username_ids = {}

for post_id in np.unique(user_answer_df['post_id'].values):
    output_file.write("  node\n")
    output_file.write("  [\n")
    output_file.write("    id " + str(post_id) + "\n")
    output_file.write("    label \"" + "post " + str(post_id) + "\"" + "\n")
    output_file.write("  ]\n")
    
for username in np.unique(user_answer_df['username'].values):
    output_file.write("  node\n")
    output_file.write("  [\n")
    output_file.write("    id " + str(id_count) + "\n")
    output_file.write("    label \"" + username + "\"" + "\n")
    output_file.write("  ]\n")
    
    username_ids[username] = id_count
    id_count += 1


# print all edges
for keys in consolidated_user_answer.keys():
    output_file.write("  edge\n")
    output_file.write("  [\n")
    output_file.write("    source " + str(username_ids[keys[0]]) + "\n")
    output_file.write("    target " + str(keys[1]) + "\n")
    output_file.write("    value " + str(consolidated_user_answer[keys]) + "\n")
    output_file.write("  ]\n")

output_file.write("]\n")
output_file.close()